In [ ]:
using Revise
using CUDA
using DataFrames
using JSON
using Statistics
using Juliana

# Config

In [ ]:
gantry_angle = 36f0
couch_angle = 64f0
nozzle_extraction = 15f0
preabsorber = "OUT"

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.9.jar";

In [ ]:
output_dir = "../output/test/dose_calc/single_spot"

mkpath(output_dir)

# Load data

In [ ]:
# ct, target = Juliana.build_water_slab()
# ct_path = "$(output_dir)/ct.dat"
# Juliana.write_ct_dat_file(ct_path, ct)

# patient_ID = "bellotti_r_water_phantom";

In [ ]:
patient_ID = "test_00"
data_dir = "/data/user/bellotti_r/data"
ct_path, patient = Juliana.load_patient_data(data_dir, patient_ID)

ct = patient.ct
target = patient.structures["PTV2=CTV2+5MM"];

In [ ]:
optimisation_points = Juliana.mask_to_points(
    ct.grid,
    target.mask,
)

optimisation_grid = Juliana.get_optimisation_grid(
    optimisation_points',
    ct.grid,
)

# Calculate dose using Fiona

In [ ]:
# Dummy value, we don't optimise...
target_dose = 1

In [ ]:
main_config = Juliana.FionaStandalone.MainConfig(
    ct_path,
    output_dir,
    target_dose,
    fiona_standalone_bin_path,
    doseResolution=ct.grid.spacing[1],
)

config_target = Juliana.FionaStandalone.StructureDefinition(target, 0)

optimisation_config = Juliana.FionaStandalone.OptimizationSettings(
    target_dose,
    0.9*target_dose,
    config_target,
    Vector{Juliana.FionaStandalone.StructureConstraints}(undef, 0),
    Juliana.FionaStandalone.to_OptimizationGrid(optimisation_grid),
)

spot_placement_config = Juliana.FionaStandalone.SpotPlacementConfig(
    [gantry_angle],
    [couch_angle],
    [nozzle_extraction],
    preabsorber,
    target,
)

target_com = vec(mean(target.points, dims=1))

field_center = Dict(
    "x" => target_com[1],
    "y" => target_com[2],
    "z" => target_com[3],
)

w0 = 1010.
E = 100000.f0
# w0 = 1e5
# E = 70000

spots = [Juliana.FionaStandalone.Spot(
    0,
    0.,
    0.,
    # The following values are obtained from running a Fiona Standalone spot placement and optimisation.
    w0,      # weight
    E, # energy [keV]
    preabsorber == "OUT" ? 0 : 1,
)]

w = [w0]

field = Juliana.FionaStandalone.FieldDefinition(
    "F0",
    0,
    gantry_angle,
    couch_angle,
    nozzle_extraction,
    field_center,
    spots,
)

plan = Juliana.FionaStandalone.TreatmentPlan(
    [field],
    target_dose,
);

In [ ]:
optimisationPointsPath = "$(output_dir)/custom_optimization_points.txt"
Juliana.FionaStandalone.write_optimisation_points(
    optimisationPointsPath,
    optimisation_points',
    optimisation_grid,
)

In [ ]:
Juliana.FionaStandalone.run_optimization(
    fiona_jar_path,
    output_dir,
    true,  # log Dij
    false, # log WED
    main_config,
    optimisation_config,
    spot_placement_config,
    plan,
    optimizationPointsPath=optimisationPointsPath,
)
Dij_fiona = Juliana.FionaStandalone.load_Dij("$output_dir/dij_matrix.dat", size(optimisation_points, 2)) * w0;

In [ ]:
dose_fiona = zeros(Float32, Tuple(ct.grid.size));

for (i, p) in enumerate(eachrow(optimisation_points'))
    indices = Juliana.xyz_to_index(p, ct.grid)
    dose_fiona[indices...] = Dij_fiona[i]
end

# Calculate dose using Juliana

## Calculate WED

In [ ]:
# Calculate dose using Juliana

huToSpPathFile = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin/huToSp.json"

hu_to_sp_dict = nothing
open(huToSpPathFile) do file
    global hu_to_sp_dict = JSON.parse(file)
end

@assert hu_to_sp_dict["dz"] == 1
@assert hu_to_sp_dict["z0"] == -1000

function convert_to_sp(value)
    sp_index = convert(
        Int64,
        round((value - hu_to_sp_dict["z0"]) / hu_to_sp_dict["dz"]),
    ) + 1
    return hu_to_sp_dict["densities"][sp_index]
end;

densities = convert_to_sp.(ct.data);

In [ ]:
direction = Juliana.angles_to_direction(gantry_angle, couch_angle)

In [ ]:
d_wed = cu(Vector{Float32}(undef, size(Dij_fiona, 1)))
d_densities = cu(densities)
d_grid = cu(ct.grid)
d_optim_points = cu(optimisation_points)
d_direction = cu(direction)

event = Juliana.calculate_wed_simple(
    d_wed,
    d_densities,
    d_grid,
    d_optim_points,
    d_direction,
    ndrange=size(Dij_fiona),
)
wait(event);

In [ ]:
event = Juliana.calculate_wed_simple(
    d_wed,
    d_densities,
    d_grid,
    d_optim_points,
    d_direction,
    ndrange=size(Dij_fiona),
)
@time wait(event);

## Load spot properties for which to calculate the dose

In [ ]:
plan = Juliana.FionaStandalone.read_plan_file("$output_dir/result_plan.json")
field = plan.fields[1]
df = DataFrame(field.spots);
d_spots_t = cu(df.t)
d_spots_u = cu(df.u)
d_spots_energy = cu(df.energykeV)
d_spots_absorbers = cu(df.numberOfAbsorbers)
energykeVSpots = df.energykeV
numberOfAbsorbersSpots = df.numberOfAbsorbers;
fieldCenter = convert.(Float32, [
    field.fieldCenter["x"],
    field.fieldCenter["y"],
    field.fieldCenter["z"],
])

## Allocate memory

In [ ]:
N = size(optimisation_points, 2)
M = length(d_spots_u)

Dij_juliana = cu(ones(N, length(d_spots_u))*NaN32);

In [ ]:
d_optimisation_points = cu(collect(optimisation_points'))
d_field_center = cu(fieldCenter)

## Load machine parameters

In [ ]:
depth_dose_curves, sigma_mcs_curves, phase_space_no_preabsorber, phase_space_with_preabsorber = Juliana.load_machine_parameters(fiona_standalone_bin_path, nozzle_extraction)

d_depth_dose_curves = cu(depth_dose_curves)
d_sigma_mcs_curves = cu(sigma_mcs_curves)
d_phase_space_no_preabsorber = cu(phase_space_no_preabsorber);
d_phase_space_with_preabsorber = cu(phase_space_with_preabsorber);

## Calculate the dose

In [ ]:
using BenchmarkTools

In [ ]:
# @benchmark Juliana.runDijKernel(
Juliana.runDijKernel(
    Dij_juliana,
    d_wed,
    cu(fieldCenter),
    gantry_angle * π / 180f0,
    couch_angle * π / 180f0,
    d_spots_t,
    d_spots_u,
    d_spots_energy,
    d_spots_absorbers,
    cu(optimisation_points'),
    d_depth_dose_curves,
    d_sigma_mcs_curves,
    d_phase_space_no_preabsorber,
    d_phase_space_with_preabsorber,
)

In [ ]:
dose_juliana = zeros(Float32, Tuple(ct.grid.size));

tmp = collect(Dij_juliana * w0)

for (i, p) in enumerate(eachrow(optimisation_points'))
    indices = Juliana.xyz_to_index(p, ct.grid)
    dose_juliana[indices...] = tmp[i]
end

# Plot

In [ ]:
maximum(abs.(dose_fiona .- dose_juliana))

In [ ]:
sum(dose_fiona)

In [ ]:
sum(dose_juliana)

In [ ]:
maximum(dose_juliana)

In [ ]:
normalisation_dose = maximum(dose_fiona)

In [ ]:
Juliana.plot_distributions(
    ct,
    [("Fiona", dose_fiona), ("Juliana", dose_juliana)],
    [],
    1, 100,
    1, 100,
    12,
#     100, 160,
#     100, 160,
#     39,
    normalisation_dose,
    Juliana.build_colorscheme(),
)

In [ ]:
maximum(abs.(dose_fiona .- dose_juliana)) / maximum(dose_fiona) * 100